# Откуда берутся датасеты? Практический проект по сбору данных и работе с текстами

<b>Цель.</b> В этом домашнем задании вам предстоит обойти все ловушки серверов, пробраться сквозь страницы html-код, собрать себе свой собственный датасет и натренировать на нём модель.

<b>Описание/Пошаговая инструкция выполнения домашнего задания:</b>

<b>Часть 1. Парсинг.</b>
</br>По аналогии с занятием, возьмите интересующий вас сайт, на котором можно пособирать какие-то данные (и при этом API не предоставляется).
</br>Напишите свой парсер, который будет бегать по страничкам и автоматически что-то собирать.
Не забывайте, что парсинг - это ответственное мероприятие, поэтому не бомбардируйте несчастные сайты слишком частыми запросами (можно ограничить число запросов в секунду при помощи time.sleep(0.3), вставленного в теле цикла).

<b>Часть 2. NLP.</b>
1. Разбейте собранные данные на train/test, отложив 20-30% наблюдений для тестирования.
2. Примените tf-idf преобразование для текстового описания. Используйте как отдельные токены, так и биграммы, отсейте стоп-слова, а также слова, которые встречаются слишком редко или слишком часто (параметры min/max_df), не забудьте убрать l2 регуляризацию, которая по умолчанию включена.
3. Если в вашем датасете целевая переменная непрерывная (например, среднее число просмотров в день), то воспользуйтесь линейной регрессией, если дискретная (положительный/отрицательный отзыв), то логистической.
4. Постройте регрессию с настройкой параметра регуляризации, оцените качество при помощи соответствующих задаче метрик.
5. Визуализируйте получившиеся коэффициенты регрессии (возьмите топ-50 слов).
6. Проинтерпретируйте результаты.

# Выполнение

Будем собирать отзывы и оценку по 5ти бальной шкале с сайта https://irecommend.ru/ по теме "Банки и банковские продукты".
</br>Первая страница банковских продуктов: https://irecommend.ru/catalog/rating/22
</br>Начиная со второй страницы ссылка выглядит так: https://irecommend.ru/catalog/rating/22?page=1
</br>Переберём все страницы с банковскими продуктами до тех пор пока не получим список всех ссылок на отзыв (пока не получим пустой массив или не наткнёмся на ответ 302 Found). </br>Ссылки будем получать из тега с классом class="read-all-reviews-link-bottom read-all-reviews-link".

Импортируем основные библиотеки.

In [1]:
import os
import time
import json
import urllib3
from urllib.parse import urljoin
import requests
from bs4 import BeautifulSoup
import pandas as pd
import random
from concurrent.futures import ThreadPoolExecutor
from fake_useragent import UserAgent

# Отключить предупреждения о непроверенных HTTPS-запросах
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def load_proxies(file_path):
    proxies = []
    with open(file_path, 'r') as file:
        lines = file.readlines()[1:]  # Пропустить заголовок
        for line in lines:
            ip, port = line.strip().split(',')
            proxies.append(f"{ip}:{port}")
    return proxies

def load_user_agents(file_path):
    user_agents = []
    with open(file_path, 'r') as file:
        lines = file.readlines()[1:]  # Пропустить заголовок
        for line in lines:
            user_agents.append(line.strip())
    return user_agents

proxies = load_proxies('proxies.csv')
user_agents = load_user_agents('user_agents.csv')

In [2]:
# Вывод длины массива
print("Количество прокси:", len(proxies))

# Вывод первых 10 записей массива
print("Вывод первых 10 прокси:")
for link in proxies[:10]:
    print(link)

Количество прокси: 300
Вывод первых 10 прокси:
50.168.163.178:80
50.221.74.130:80
50.168.72.115:80
50.174.7.162:80
50.172.75.121:80
50.222.245.44:80
50.217.226.43:80
50.171.122.30:80
68.185.57.66:80
50.168.72.122:80


In [3]:
# Вывод длины массива
print("Количество user_agents:", len(user_agents))

# Вывод первых 10 записей массива
print("Вывод первых 10 user_agents:")
for link in user_agents[:10]:
    print(link)

Количество user_agents: 300
Вывод первых 10 user_agents:
"Mozilla/5.0 (Linux; Android 8.0.0; SM-G935F Build/R16NW; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/116.0.0.0 Mobile Safari/537.36"
"Mozilla/5.0 (Linux; Android 12; SM-A035M Build/SP1A.210812.016; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/108.0.5359.128 Mobile Safari/537.36"
"TuneIn Radio Pro/26.2.1; iPhone13,2; iOS/16.6.1"
"Mozilla/5.0 (Linux; Android 12; SM-A315G Build/SP1A.210812.016; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/116.0.0.0 Mobile Safari/537.36 Instagram 300.0.0.29.110 Android (31/12; 420dpi; 1080x2195; samsung; SM-A315G; a31; mt6768; it_IT; 515103471)"
"Mozilla/5.0 (Linux; Android 12; LM-Q920N Build/SKQ1.211103.001; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/96.0.4664.104 Mobile Safari/537.36"
"Mozilla/5.0 (Linux; Android 10; MI 8 Build/QKQ1.190828.002; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/117.0.0.0 Mobile Safar

In [4]:
# Отключиv предупреждения о непроверенных HTTPS-запросах
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def get_review_links(session, page_url, headers):
    try:
        response = session.get(page_url, headers=headers, verify=False)
        if response.status_code == 302:  # Перенаправление указывает на конец страницы
            return []
        soup = BeautifulSoup(response.content, 'html.parser')
        review_links = []
        for link in soup.find_all('a', class_='read-all-reviews-link-bottom read-all-reviews-link'):
            review_links.append(urljoin(page_url, link.get('href')))
        return review_links
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return []

def scrape_all_review_links(base_url, start_page=0):
    ua = UserAgent()
    all_links = []
    page = start_page
    with requests.Session() as session:
        while True:
            headers = {'User-Agent': ua.random}
            if page == 0:
                page_url = base_url
            else:
                page_url = f"{base_url}?page={page}"
            links = get_review_links(session, page_url, headers)
            if not links:  # Если ссылки не найдены, прерываем цикл
                break
            all_links.extend(links)
            page += 1
            time.sleep(0.3)  # Задержка между запросами
    return all_links

def save_links_to_file(links, filename):
    with open(filename, 'w') as f:
        json.dump(links, f)

def load_links_from_file(filename):
    try:
        with open(filename, 'r') as f:
            return json.load(f)
    except FileNotFoundError:
        return []

In [5]:
# Базовый URL-адрес для первоначального списка отзывов
base_url = "https://irecommend.ru/catalog/rating/22"

# Загрузим все ссылки на просмотр с начальных страниц
all_review_links = load_links_from_file('all_review_links.json')
if not all_review_links:
    all_review_links = scrape_all_review_links(base_url)
    save_links_to_file(all_review_links, 'all_review_links.json')

# Вывод длины массива
print("Количество ссылок:", len(all_review_links))

# Вывод первых 10 записей массива
print("Вывод первых 10 ссылок:")
for link in all_review_links[:10]:
    print(link)

Количество ссылок: 340
Вывод первых 10 ссылок:
https://irecommend.ru/content/vsegda-da
https://irecommend.ru/content/detskaya-karta-tinkoff-junior
https://irecommend.ru/content/alfa-bank-premium
https://irecommend.ru/content/tinkoff-drive-debetovaya-karta-drive
https://irecommend.ru/content/nakopitelnyi-alfa-schet-1
https://irecommend.ru/content/debetovaya-karta-tinkoff-lamoda
https://irecommend.ru/content/ipoteka-tinkoff-0
https://irecommend.ru/content/programma-urozhai-ot-rosselkhozbanka
https://irecommend.ru/content/kredit-nalichnymi-ot-tinkoff-bank-1
https://irecommend.ru/content/kreditnaya-karta-tinkoff-all-airlines-0


In [6]:
def get_review_snippet_links(session, product_url, headers):
    try:
        response = session.get(product_url, headers=headers, verify=False)
        if response.status_code != 200:
            return []
        soup = BeautifulSoup(response.content, 'html.parser')
        snippet_links = []
        for link in soup.find_all('a', class_='reviewTextSnippet'):
            snippet_links.append(urljoin(product_url, link.get('href')))
        return snippet_links
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return []

def scrape_all_snippet_links(all_review_links):
    usag = UserAgent()
    all_snippet_links = []
    with requests.Session() as session:
        for product_link in all_review_links:
            headers = {'User-Agent': usag.random}
            snippet_links = get_review_snippet_links(session, product_link, headers)
            all_snippet_links.extend(snippet_links)
            time.sleep(0.3)
    return all_snippet_links

In [7]:
all_snippet_links = load_links_from_file('all_snippet_links.json')
if not all_snippet_links:
    all_snippet_links = scrape_all_snippet_links(all_review_links)
    save_links_to_file(all_snippet_links, 'all_snippet_links.json')

print("Количество ссылок:", len(all_snippet_links))

print("Вывод первых 10 ссылок:")
for link in all_snippet_links[:10]:
    print(link)

Количество ссылок: 1061
Вывод первых 10 ссылок:
https://irecommend.ru/content/poidi-tuda-neznamo-kuda-i-prinesi-neznamo-chto-ili-istoriya-o-tom-kak-ne-zashchishcheny-nash
https://irecommend.ru/content/zachem-rebenku-bankovskaya-karta-s-tinkoff-junior-deti-uchatsya-ne-tolko-tratit-dengi-no-i-z
https://irecommend.ru/content/kartu-rekomenduyu-poskolku-v-ee-nalichii-bolshe-plyusov-chem-minusov-i-blagodarya-besplatnom
https://irecommend.ru/content/detskaya-karta-tinkoff-junior-sovremennyi-i-udobnyi-sposob-vydachi-karmannykh-deneg-rebenku
https://irecommend.ru/content/dva-v-odnom-karta-i-mini-shkola-finansovoi-gramotnosti
https://irecommend.ru/content/karta-blagodarya-kotoroi-rebenok-s-detstva-uchitsya-upravlyat-finansami-vse-po-vzroslomu-poc
https://irecommend.ru/content/klassnaya-karta-no-est-nyuans
https://irecommend.ru/content/syn-teper-ne-nosit-meloch-i-poluchaet-dop-dengi-za-vypolnennye-zadaniya-i-glavnoe-ya-vsegda
https://irecommend.ru/content/banku-bezrazlichno-ponyatie-detskaya-psik

In [ ]:
def get_review_snippet_links(session, product_url, headers):
    try:
        response = session.get(product_url, headers=headers, verify=False)
        if response.status_code != 200:
            return []
        soup = BeautifulSoup(response.content, 'html.parser')
        snippet_links = []
        for link in soup.find_all('a', class_='reviewTextSnippet'):
            snippet_links.append(urljoin(product_url, link.get('href')))
        return snippet_links
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return []

def scrape_all_snippet_links(all_review_links):
    usag = UserAgent()
    all_snippet_links = []
    with requests.Session() as session:
        for product_link in all_review_links:
            headers = {'User-Agent': usag.random}
            snippet_links = get_review_snippet_links(session, product_link, headers)
            all_snippet_links.extend(snippet_links)
            time.sleep(0.3)
    return all_snippet_links

In [ ]:
all_snippet_links = load_links_from_file('all_snippet_links.json')
if not all_snippet_links:
    all_snippet_links = scrape_all_snippet_links(all_review_links)
    save_links_to_file(all_snippet_links, 'all_snippet_links.json')

print("Количество ссылок:", len(all_snippet_links))

print("Вывод первых 10 ссылок:")
for link in all_snippet_links[:10]:
    print(link)

In [ ]:
def get_random_proxy():
    return random.choice(proxies)

def get_random_user_agent():
    return random.choice(user_agents)

def get_review_links(session, page_url, headers, proxy):
    try:
        response = session.get(page_url, headers=headers, proxies={"http": proxy, "https": proxy}, verify=False)
        if response.status_code == 302:  # Перенаправление указывает на конец страницы
            return []
        soup = BeautifulSoup(response.content, 'html.parser')
        review_links = []
        for link in soup.find_all('a', class_='read-all-reviews-link-bottom read-all-reviews-link'):
            review_links.append(urljoin(page_url, link.get('href')))
        return review_links
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return []

def save_links_to_file(links, filename):
    with open(filename, 'w') as f:
        json.dump(links, f)

def get_review_snippet_links(session, product_url, headers, proxy):
    try:
        response = session.get(product_url, headers=headers, proxies={"http": proxy, "https": proxy}, verify=False)
        if response.status_code != 200:
            return []
        soup = BeautifulSoup(response.content, 'html.parser')
        snippet_links = []
        for link in soup.find_all('a', class_='reviewTextSnippet'):
            snippet_links.append(urljoin(product_url, link.get('href')))
        return snippet_links
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return []

def scrape_snippet_links_for_product(session, product_link):
    headers = {'User-Agent': get_random_user_agent()}
    proxy = get_random_proxy()
    return get_review_snippet_links(session, product_link, headers, proxy)

def scrape_all_snippet_links(all_review_links):
    all_snippet_links = []
    with requests.Session() as session:
        for product_link in all_review_links:
            snippet_links = scrape_snippet_links_for_product(session, product_link)
            all_snippet_links.extend(snippet_links)
            time.sleep(random.uniform(0.5, 2.0))  # Случайная задержка
    return all_snippet_links

In [ ]:
def load_links_from_file(filename):
    try:
        with open(filename, 'r') as f:
            return json.load(f)
    except FileNotFoundError:
        return []

In [ ]:
def scrape_all_review_links(base_url, start_page=0):
    all_links = []
    page = start_page
    with requests.Session() as session:
        while True:
            headers = {'User-Agent': get_random_user_agent()}
            proxy = get_random_proxy()
            if page == 0:
                page_url = base_url
            else:
                page_url = f"{base_url}?page={page}"
            links = get_review_links(session, page_url, headers, proxy)
            if not links:  # Если ссылки не найдены, прерываем цикл
                break
            all_links.extend(links)
            page += 1
            time.sleep(random.uniform(0.3, 0.5))  # Случайная задержка между запросами
    return all_links

In [ ]:
# Базовый URL-адрес для первоначального списка отзывов
base_url = "https://irecommend.ru/catalog/rating/22"

# Загрузим все ссылки на просмотр с начальных страниц
all_review_links = load_links_from_file('all_review_links.json')
if not all_review_links:
    all_review_links = scrape_all_review_links(base_url)
    save_links_to_file(all_review_links, 'all_review_links.json')

# Вывод длины массива
print("Количество ссылок:", len(all_review_links))

# Вывод первых 10 записей массива
print("Вывод первых 10 ссылок:")
for link in all_review_links[:10]:
    print(link)

In [ ]:
def scrape_review_data_for_link(session, review_link, max_retries):
    retries = 0
    while retries < max_retries:
        headers = {
            'User-Agent': get_random_user_agent(),
            'Accept-Language': 'en-US,en;q=0.9',
            'Accept-Encoding': 'gzip, deflate, br',
            'Referer': review_link
        }
        proxy = get_random_proxy()
        try:
            response = session.get(review_link, headers=headers, proxies={"http": proxy, "https": proxy}, verify=False)
            if response.status_code == 521:
                print(f"Server error on {review_link}, status code: 521. Retrying...")
                retries += 1
                time.sleep(random.uniform(0.5, 2.0))
                continue
            if response.status_code != 200:
                print(f"Failed to load page: {review_link}, status code: {response.status_code}")
                break
            soup = BeautifulSoup(response.content, 'html.parser')

            # Получаем текст отзыва
            review_div = soup.find('div', class_='description hasinlineimage')
            if not review_div:
                print(f"No review text found on: {review_link}")
                break
            review_text = " ".join(p.get_text(separator=' ', strip=True) for p in review_div.find_all('p'))

            # Получаем рейтинг
            rating_meta = soup.find('meta', itemprop='ratingValue')
            if not rating_meta:
                print(f"No rating found on: {review_link}")
                break
            rating_value = rating_meta.get('content')

            print(f"Scraped review: {review_text[:30]}..., Rating: {rating_value}")
            return {'text_review': review_text, 'rating_review': rating_value}
        except requests.exceptions.RequestException as e:
            print(f"Request failed: {e}")
            retries += 1
            time.sleep(random.uniform(0.5, 2.0))
    return None

In [ ]:
# Базовый URL-адрес для первоначального списка отзывов
base_url = "https://irecommend.ru/catalog/rating/22"

# Загрузим все ссылки на просмотр с начальных страниц
all_review_links = load_links_from_file('all_review_links.json')
if not all_review_links:
    all_review_links = scrape_all_review_links(base_url)
    save_links_to_file(all_review_links, 'all_review_links.json')

# Вывод длины массива
print("Количество ссылок:", len(all_review_links))

# Вывод первых 10 записей массива
print("Вывод первых 10 ссылок:")
for link in all_review_links[:10]:
    print(link)

In [ ]:
# Загрузим все ссылки на сниппеты отзывов
all_snippet_links = load_links_from_file('all_snippet_links.json')
if not all_snippet_links:
    all_snippet_links = scrape_all_snippet_links(all_review_links)
    save_links_to_file(all_snippet_links, 'all_snippet_links.json')

print("Количество ссылок:", len(all_snippet_links))

print("Вывод первых 10 ссылок:")
for link in all_snippet_links[:10]:
    print(link)

In [ ]:
def scrape_review_data(all_snippet_links, max_retries=2):
    reviews_data = []
    with requests.Session() as session:
        for review_link in all_snippet_links:
            review_data = scrape_review_data_for_link(session, review_link, max_retries)
            if review_data:
                reviews_data.append(review_data)
            time.sleep(random.uniform(0.5, 2.0))
    return reviews_data

In [ ]:
# Сбор данных отзывов
filename = 'reviews_dataset.csv'

if os.path.exists(filename):
    df = pd.read_csv(filename)
    if df.empty:
        print("Файл 'reviews_dataset.csv' пуст.")
    else:
        print("Данные успешно загружены из файла 'reviews_dataset.csv'.")
else:
    print("В файле 'reviews_dataset.csv' не найдено никаких данных, пытаюсь восстановить данные отзывов...")
    reviews_data = scrape_review_data(all_snippet_links)
    df = pd.DataFrame(reviews_data)
    df.to_csv(filename, index=False)
    print("Данные успешно обработаны и сохранены в файле 'reviews_dataset.csv'.")

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
# Удаление строк с пустыми значениями в колонке text_review
df = df.dropna(subset=['text_review'])
df.info()

In [ ]:
# Подсчет количества значений по классам в признаке "rating_review"
rating_counts = df['rating_review'].value_counts().sort_index()

# Построение графика
plt.figure(figsize=(10, 6))
plt.bar(rating_counts.index, rating_counts.values, color='skyblue')
plt.xlabel('Rating Review')
plt.ylabel('Count')
plt.title('Number of Values by Classes in "rating_review"')
plt.xticks(rotation=0)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Показать график
plt.show()